In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
book_df = spark.read.csv('gs://process-data-bucket/output/book_data.csv',header=True,inferSchema=True)
transaction_df = spark.read.csv('gs://process-data-bucket/output/transaction_data.csv',header=True,inferSchema=True)
conversion_df = spark.read.csv('gs://process-data-bucket/output/conversion_rate_data.csv',header=True,inferSchema=True)

In [ ]:
#change col name to remove after join
book_df = book_df.withColumnRenamed('Book_ID','join_temp')

df = transaction_df.join(book_df,transaction_df.book_id==book_df.join_temp,"left")
df = df.drop('join_temp')
df = df.withColumn('date',to_date('timestamp'))
df.show()

In [ ]:
temp = conversion_df
temp = temp.withColumn('date',to_date('date'))
temp = temp.withColumnRenamed('date','join_temp')
temp.show()

In [ ]:
df = df.join(temp,df.date==temp.join_temp,"left")
df = df.drop('join_temp')
df.show()

In [ ]:
# remove $ and change data type to float
df = df.withColumn('Price', regexp_replace('Price', '^\W{1}', ''))
df = df.withColumn('Price',df.Price.cast('float'))
df.show()

In [13]:
# Create THB column
df.withColumn('THBPrice',df.Price*df.conversion_rate).show()
# drop date column
df = df.drop('date')

In [ ]:
df.write.csv('gs://process-data-bucket/output/audible_data.csv', header = True)

In [ ]:
df.dtypes